In [1]:
from pathlib import Path
import os, sys, platform, torch

PROJECT   = Path("/scratch/qin.yife/Generative_Project")
CSV_CLEAN = PROJECT/"data"/"processed"/"pairs.clean.csv"
HF_HOME   = PROJECT/".cache"/"huggingface"
OUT_DIR   = PROJECT/"outputs"/"m2"; OUT_DIR.mkdir(parents=True, exist_ok=True)
LOG_CSV   = PROJECT/"metrics"/"m2_runs.csv"; LOG_CSV.parent.mkdir(parents=True, exist_ok=True)

os.environ["HF_HOME"] = str(HF_HOME)
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
print("CSV exists:", CSV_CLEAN.exists())
print("HF_HOME:", os.environ["HF_HOME"])
print("device:", "cuda" if torch.cuda.is_available() else "cpu", "| torch:", torch.__version__, "| python:", sys.version.split()[0], "|", platform.platform())
print("OUT_DIR:", OUT_DIR)
print("LOG_CSV:", LOG_CSV)

CSV exists: True
HF_HOME: /scratch/qin.yife/Generative_Project/.cache/huggingface
device: cpu | torch: 2.9.0+cpu | python: 3.12.4 | Linux-5.14.0-362.13.1.el9_3.x86_64-x86_64-with-glibc2.34
OUT_DIR: /scratch/qin.yife/Generative_Project/outputs/m2
LOG_CSV: /scratch/qin.yife/Generative_Project/metrics/m2_runs.csv


In [2]:
import pandas as pd
from pathlib import Path

# Paths
PROJECT    = Path("/scratch/qin.yife/Generative_Project")
CSV_CLEAN  = PROJECT / "data" / "processed" / "pairs.clean.csv"
OUT_DIR    = PROJECT / "outputs" / "m2"
PROMPT_TXT = OUT_DIR / "eval_prompts.txt"

# Load captions
df = pd.read_csv(CSV_CLEAN)
df["caption"] = df["caption"].astype(str).str.strip()

# Basic filtering: non-empty, reasonable length
df = df[(df["caption"].str.len() >= 16) & (df["caption"].str.len() <= 120)].drop_duplicates(subset=["caption"])

# Deterministic sample of 8
n = min(8, len(df))
eval_prompts = df.sample(n=n, random_state=42)["caption"].tolist()

# Save for reproducibility
OUT_DIR.mkdir(parents=True, exist_ok=True)
with open(PROMPT_TXT, "w", encoding="utf-8") as f:
    for p in eval_prompts:
        f.write(p + "\n")

print("Saved eval prompts ->", PROMPT_TXT)
for i, p in enumerate(eval_prompts, 1):
    print(f"{i:02d}. {p}")

Saved eval prompts -> /scratch/qin.yife/Generative_Project/outputs/m2/eval_prompts.txt
01. Two gray dogs run near the water on a manicured lawn in the fall .
02. Man in black wetsuit on his surfboard riding a blue wave with the beach and onlookers in the background .
03. Many people are clustered closely together , and several of them have alcohol .
04. A man in a suit stands next to a white carriage pulled by two white horses while a bride sits inside .
05. A man in a white uniform rides a motocross bike down a forest trail .
06. A band performs a song with a saxophone , guitars , base , drums and a keyboard .
07. A low angled shot of skateboarder in midair while doing a trick , with the blue sky and white clouds in the background .
08. Long-necked , flying white bird grazes water with black legs .


In [3]:
import sys, subprocess, os

def ensure_diffusers_stack():
    """
    Ensure diffusers + accelerate + transformers + safetensors are available.
    If import fails, install into a scratch-local site-packages and append to sys.path.
    """
    try:
        import diffusers, transformers
        import torch
        print("Already available -> diffusers:", diffusers.__version__,
              "| transformers:", transformers.__version__,
              "| torch:", torch.__version__)
        return
    except Exception as e:
        print("Import failed, will install to scratch site-packages:", e)

    target = "/scratch/qin.yife/Generative_Project/.python_pkgs"
    os.makedirs(target, exist_ok=True)
    print("Installing into:", target)

    # Install pinned versions compatible with your current setup
    cmd = [
        sys.executable, "-m", "pip", "install",
        "--target", target,
        "diffusers==0.30.0", "accelerate==0.34.2",
        "transformers==4.45.0", "safetensors"
    ]
    print(">>", " ".join(cmd))
    subprocess.check_call(cmd)

    if target not in sys.path:
        sys.path.append(target)

    import diffusers, transformers, torch
    print("Installed -> diffusers:", diffusers.__version__,
          "| transformers:", transformers.__version__,
          "| torch:", torch.__version__)

ensure_diffusers_stack()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Already available -> diffusers: 0.30.0 | transformers: 4.45.0 | torch: 2.9.0+cpu


In [4]:
import os, json, torch
from pathlib import Path
from diffusers import StableDiffusionPipeline, DDIMScheduler

# Paths
PROJECT  = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR  = PROJECT / "outputs" / "m2"
OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_META = OUT_DIR / "model_choice.json"

# Device & dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

# Preferred -> SD 1.5 (more scheduler options); Fallback -> SD Turbo (faster on CPU)
primary_model   = "runwayml/stable-diffusion-v1-5"
fallback_model  = "stabilityai/sd-turbo"

def load_pipe(model_id: str):
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=dtype,
        cache_dir=os.environ.get("HF_HOME"),
        use_safetensors=True,
    )
    # Use DDIM to start (we'll compare schedulers later)
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    # Memory-friendly options
    try:
        pipe.enable_attention_slicing()
    except Exception:
        pass
    pipe = pipe.to(device)
    return pipe

pipe = None
used_model = None
try:
    print(f"Loading primary model: {primary_model}")
    pipe = load_pipe(primary_model)
    used_model = primary_model
except Exception as e:
    print(f"[Warn] Failed to load {primary_model}: {e}")
    print(f"Falling back to: {fallback_model}")
    pipe = load_pipe(fallback_model)
    used_model = fallback_model

# Save model choice for reproducibility
with open(MODEL_META, "w") as f:
    json.dump({"used_model": used_model, "device": device, "dtype": str(dtype), "scheduler": type(pipe.scheduler).__name__}, f, indent=2)

print("Loaded model:", used_model)
print("Scheduler   :", type(pipe.scheduler).__name__)
print("Device/dtype:", device, "/", dtype)
print("Text encoder:", pipe.text_encoder.__class__.__name__)
print("Tokenizer   :", getattr(pipe, 'tokenizer', None).__class__.__name__)
print("Saved model choice ->", MODEL_META)

Loading primary model: runwayml/stable-diffusion-v1-5


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/home/qin.yife/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded model: runwayml/stable-diffusion-v1-5
Scheduler   : DDIMScheduler
Device/dtype: cpu / torch.float32
Text encoder: CLIPTextModel
Tokenizer   : CLIPTokenizer
Saved model choice -> /scratch/qin.yife/Generative_Project/outputs/m2/model_choice.json


In [5]:
import torch, csv, json
from pathlib import Path

# Paths
PROJECT   = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR   = PROJECT / "outputs" / "m2"
LOG_CSV   = PROJECT / "metrics" / "m2_runs.csv"
PROMPT_TXT= OUT_DIR / "eval_prompts.txt"
MODEL_META= OUT_DIR / "model_choice.json"

# Load eval prompts (use first 4 for a quick CPU baseline)
with open(PROMPT_TXT, "r", encoding="utf-8") as f:
    eval_prompts = [ln.strip() for ln in f if ln.strip()]
prompts = eval_prompts[:4]

# Read model choice (for logging)
try:
    used_model = json.loads(MODEL_META.read_text())["used_model"]
except Exception:
    used_model = "unknown"

# Inference params
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg    = 7.5
steps  = 15   # keep modest on CPU
seed   = 20251114
gen    = torch.Generator(device=device).manual_seed(seed)

# Generate & save
rows = []
for i, p in enumerate(prompts, 1):
    image = pipe(
        p,
        num_inference_steps=steps,
        guidance_scale=cfg,
        generator=gen
    ).images[0]
    fn = OUT_DIR / f"baseline_ddim_cfg{cfg}_s{steps}_seed{seed}_{i:02d}.png"
    image.save(fn)

    rows.append(dict(
        file=str(fn),
        prompt=p,
        scheduler=type(pipe.scheduler).__name__,
        guidance_scale=cfg,
        steps=steps,
        seed=seed,
        model=used_model
    ))

# Append to CSV log
write_header = not LOG_CSV.exists()
with open(LOG_CSV, "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    if write_header:
        writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} images to:", OUT_DIR)
for r in rows:
    print(" -", r["file"])
print("Logged to:", LOG_CSV)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Saved 4 images to: /scratch/qin.yife/Generative_Project/outputs/m2
 - /scratch/qin.yife/Generative_Project/outputs/m2/baseline_ddim_cfg7.5_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/baseline_ddim_cfg7.5_s15_seed20251114_02.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/baseline_ddim_cfg7.5_s15_seed20251114_03.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/baseline_ddim_cfg7.5_s15_seed20251114_04.png
Logged to: /scratch/qin.yife/Generative_Project/metrics/m2_runs.csv


In [7]:
import torch, csv, json
from pathlib import Path

# Paths
PROJECT    = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR    = PROJECT / "outputs" / "m2"
LOG_CSV    = PROJECT / "metrics" / "m2_runs.csv"
PROMPT_TXT = OUT_DIR / "eval_prompts.txt"
MODEL_META = OUT_DIR / "model_choice.json"

# Load prompts (first 4 to keep CPU runtime reasonable)
with open(PROMPT_TXT, "r", encoding="utf-8") as f:
    eval_prompts = [ln.strip() for ln in f if ln.strip()]
prompts = eval_prompts[:4]

# Read model choice (for logging)
try:
    used_model = json.loads(MODEL_META.read_text())["used_model"]
except Exception:
    used_model = "unknown"

# Device / inference params
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg, steps, seed = 7.5, 15, 20251114
generator = torch.Generator(device=device).manual_seed(seed)

# --- FIXED: encode with padding="max_length" on both pos & neg ---
def get_text_and_negative_embeds(texts):
    max_len = pipe.tokenizer.model_max_length

    # Positive prompts
    tok = pipe.tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt",
    ).to(device)
    with torch.no_grad():
        pos = pipe.text_encoder(**tok).last_hidden_state  # [B, max_len, 768]

    # Negative prompts (empty), same padding/max_length
    neg_tok = pipe.tokenizer(
        [""] * len(texts),
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt",
    ).to(device)
    with torch.no_grad():
        neg = pipe.text_encoder(**neg_tok).last_hidden_state  # [B, max_len, 768]

    return pos, neg

prompt_embeds, negative_embeds = get_text_and_negative_embeds(prompts)
print("prompt_embeds:", tuple(prompt_embeds.shape), "| negative_embeds:", tuple(negative_embeds.shape))
print("scheduler:", type(pipe.scheduler).__name__, "| model:", used_model, "| device:", device)

# --- Generate using precomputed embeddings (fixed shapes) ---
rows = []
for i in range(len(prompts)):
    pe = prompt_embeds[i:i+1]
    ne = negative_embeds[i:i+1]
    image = pipe(
        prompt_embeds=pe,
        negative_prompt_embeds=ne,
        num_inference_steps=steps,
        guidance_scale=cfg,
        generator=generator,
    ).images[0]

    fn = OUT_DIR / f"embedfix_ddim_cfg{cfg}_s{steps}_seed{seed}_{i+1:02d}.png"
    image.save(fn)

    rows.append(dict(
        file=str(fn),
        prompt=prompts[i],
        scheduler=type(pipe.scheduler).__name__,
        guidance_scale=cfg,
        steps=steps,
        seed=seed,
        via="prompt_embeds_maxlen",
        model=used_model,
    ))

# Append to CSV log
write_header = not LOG_CSV.exists()
with open(LOG_CSV, "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    if write_header:
        writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} images via prompt_embeds (fixed) to:", OUT_DIR)
for r in rows:
    print(" -", r["file"])
print("Logged to:", LOG_CSV)

prompt_embeds: (4, 77, 768) | negative_embeds: (4, 77, 768)
scheduler: DDIMScheduler | model: runwayml/stable-diffusion-v1-5 | device: cpu


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Saved 4 images via prompt_embeds (fixed) to: /scratch/qin.yife/Generative_Project/outputs/m2
 - /scratch/qin.yife/Generative_Project/outputs/m2/embedfix_ddim_cfg7.5_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/embedfix_ddim_cfg7.5_s15_seed20251114_02.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/embedfix_ddim_cfg7.5_s15_seed20251114_03.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/embedfix_ddim_cfg7.5_s15_seed20251114_04.png
Logged to: /scratch/qin.yife/Generative_Project/metrics/m2_runs.csv


In [8]:
import torch, csv, json
from pathlib import Path

# Paths
PROJECT    = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR    = PROJECT / "outputs" / "m2"
LOG_CSV    = PROJECT / "metrics" / "m2_runs.csv"
PROMPT_TXT = OUT_DIR / "eval_prompts.txt"
MODEL_META = OUT_DIR / "model_choice.json"

# Load prompts (use first 3 to keep CPU time reasonable)
with open(PROMPT_TXT, "r", encoding="utf-8") as f:
    eval_prompts = [ln.strip() for ln in f if ln.strip()]
prompts = eval_prompts[:3]

# Read model choice (for logging)
try:
    used_model = json.loads(MODEL_META.read_text())["used_model"]
except Exception:
    used_model = "unknown"

# Sweep settings
cfg_list = [3.0, 5.0, 7.5, 10.0, 12.0]
steps    = 15
seed     = 20251114
device   = "cuda" if torch.cuda.is_available() else "cpu"

rows = []
for cfg in cfg_list:
    for i, p in enumerate(prompts, 1):
        # Recreate generator each call to keep initial noise identical across CFG values
        gen = torch.Generator(device=device).manual_seed(seed)

        image = pipe(
            p,
            num_inference_steps=steps,
            guidance_scale=cfg,
            generator=gen
        ).images[0]

        fn = OUT_DIR / f"ddim_CFG{cfg}_s{steps}_seed{seed}_{i:02d}.png"
        image.save(fn)

        rows.append(dict(
            file=str(fn),
            prompt=p,
            scheduler=type(pipe.scheduler).__name__,
            guidance_scale=cfg,
            steps=steps,
            seed=seed,
            model=used_model,
            sweep="cfg"
        ))

# Append to CSV log
write_header = not LOG_CSV.exists()
with open(LOG_CSV, "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    if write_header:
        writer.writeheader()
    writer.writerows(rows)

print(f"Saved {len(rows)} images to:", OUT_DIR)
for r in rows[:5]:
    print(" -", r["file"])
print("... (truncated)")
print("Logged to:", LOG_CSV)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Saved 15 images to: /scratch/qin.yife/Generative_Project/outputs/m2
 - /scratch/qin.yife/Generative_Project/outputs/m2/ddim_CFG3.0_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/ddim_CFG3.0_s15_seed20251114_02.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/ddim_CFG3.0_s15_seed20251114_03.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/ddim_CFG5.0_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/ddim_CFG5.0_s15_seed20251114_02.png
... (truncated)
Logged to: /scratch/qin.yife/Generative_Project/metrics/m2_runs.csv


In [9]:
import torch, csv, json, os
from pathlib import Path
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler

# Paths
PROJECT    = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR    = PROJECT / "outputs" / "m2"; OUT_DIR.mkdir(parents=True, exist_ok=True)
LOG_CSV    = PROJECT / "metrics" / "m2_runs.csv"
PROMPT_TXT = OUT_DIR / "eval_prompts.txt"
MODEL_META = OUT_DIR / "model_choice.json"

# Load prompts (first 2 to keep CPU runtime reasonable)
with open(PROMPT_TXT, "r", encoding="utf-8") as f:
    eval_prompts = [ln.strip() for ln in f if ln.strip()]
prompts = eval_prompts[:2]

# Read model choice (for logging)
try:
    used_model = json.loads(MODEL_META.read_text())["used_model"]
except Exception:
    used_model = "unknown"

# Settings
steps = 15
cfg   = 7.5
seed  = 20251114
device = "cuda" if torch.cuda.is_available() else "cpu"

def run_with_scheduler(sched_cls, tag: str):
    # Switch scheduler
    pipe.scheduler = sched_cls.from_config(pipe.scheduler.config)
    rows = []
    for i, p in enumerate(prompts, 1):
        gen = torch.Generator(device=device).manual_seed(seed)  # fixed noise for fair comparison
        image = pipe(
            p, num_inference_steps=steps, guidance_scale=cfg, generator=gen
        ).images[0]
        fn = OUT_DIR / f"{tag}_CFG{cfg}_s{steps}_seed{seed}_{i:02d}.png"
        image.save(fn)
        rows.append(dict(
            file=str(fn),
            prompt=p,
            scheduler=type(pipe.scheduler).__name__,
            guidance_scale=cfg,
            steps=steps,
            seed=seed,
            model=used_model,
            sweep="scheduler"
        ))
    return rows

rows = []
rows += run_with_scheduler(DDIMScheduler,                   "DDIM")
rows += run_with_scheduler(EulerAncestralDiscreteScheduler, "EulerA")
rows += run_with_scheduler(DPMSolverMultistepScheduler,     "DPMSolver")

# Append to CSV log
write_header = not LOG_CSV.exists()
with open(LOG_CSV, "a", newline="") as f:
    w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    if write_header: w.writeheader()
    w.writerows(rows)

print(f"Saved {len(rows)} images across schedulers to:", OUT_DIR)
for r in rows:
    print(" -", r["file"])
print("Logged to:", LOG_CSV)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Saved 6 images across schedulers to: /scratch/qin.yife/Generative_Project/outputs/m2
 - /scratch/qin.yife/Generative_Project/outputs/m2/DDIM_CFG7.5_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/DDIM_CFG7.5_s15_seed20251114_02.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/EulerA_CFG7.5_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/EulerA_CFG7.5_s15_seed20251114_02.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/DPMSolver_CFG7.5_s15_seed20251114_01.png
 - /scratch/qin.yife/Generative_Project/outputs/m2/DPMSolver_CFG7.5_s15_seed20251114_02.png
Logged to: /scratch/qin.yife/Generative_Project/metrics/m2_runs.csv


In [11]:
from pathlib import Path
import shutil, csv, json

# Paths
PROJECT    = Path("/scratch/qin.yife/Generative_Project")
OUT_DIR    = PROJECT / "outputs" / "m2"
SUBMIT_DIR = OUT_DIR / "submit"
LOG_CSV    = PROJECT / "metrics" / "m2_runs.csv"
MODEL_META = OUT_DIR / "model_choice.json"
SUMMARY_MD = OUT_DIR / "M2_summary_template.md"

SUBMIT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Pick up to 10 most recent PNGs (ensure at least 5 if possible)
pngs = sorted([p for p in OUT_DIR.glob("*.png")], key=lambda p: p.stat().st_mtime, reverse=True)
if not pngs:
    raise RuntimeError("No PNG files found in OUT_DIR.")
selected = pngs[:10] if len(pngs) >= 10 else pngs
print(f"Selected {len(selected)} images (most recent) for submission.")

# 2) Copy into SUBMIT_DIR with ordered names m2_01.png, m2_02.png, ...
copied = []
for i, src in enumerate(selected, 1):
    dst = SUBMIT_DIR / f"m2_{i:02d}.png"
    shutil.copy2(src, dst)
    copied.append((src, dst))
print("Copied to:", SUBMIT_DIR)
for src, dst in copied[:5]:
    print(" -", dst.name, "<-", src.name)
if len(copied) > 5:
    print(" ...")

# 3) Robustly read log CSV (tolerate inconsistent columns)
def load_log_rows(path: Path):
    rows = []
    if not path.exists():
        return rows
    with open(path, "r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f, restkey="__extra__", restval="")
        for r in reader:
            rows.append(r)
    return rows

log_rows = load_log_rows(LOG_CSV)
index_by_basename = {}
for r in log_rows:
    fpath = (r.get("file") or "").strip()
    if fpath:
        index_by_basename[Path(fpath).name] = r

# 4) Build manifest with normalized columns
manifest_cols = ["submit_name","file","prompt","scheduler","guidance_scale","steps","seed","model"]
manifest_path = SUBMIT_DIR / "manifest.csv"
with open(manifest_path, "w", encoding="utf-8", newline="") as f:
    w = csv.DictWriter(f, fieldnames=manifest_cols)
    w.writeheader()
    for src, dst in copied:
        r = index_by_basename.get(src.name, {})
        row = {
            "submit_name": dst.name,
            "file": r.get("file", str(src)),
            "prompt": r.get("prompt", ""),
            "scheduler": r.get("scheduler", ""),
            "guidance_scale": r.get("guidance_scale", ""),
            "steps": r.get("steps", ""),
            "seed": r.get("seed", ""),
            "model": r.get("model", ""),
        }
        w.writerow(row)
print("Wrote manifest ->", manifest_path)

# 5) 1-page summary template (fill basics; you add observations)
used_model = "unknown"; device="cpu"; dtype="torch.float32"; scheduler="(varies)"
try:
    meta = json.loads(MODEL_META.read_text())
    used_model = meta.get("used_model", used_model)
    device     = meta.get("device", device)
    dtype      = meta.get("dtype", dtype)
    scheduler  = meta.get("scheduler", scheduler)
except Exception:
    pass

# Derive distinct values from log (tolerant)
cfgs, scheds, steps = set(), set(), set()
for r in log_rows:
    if r.get("guidance_scale", "") != "": cfgs.add(str(r["guidance_scale"]))
    if r.get("scheduler", "") != "":      scheds.add(str(r["scheduler"]))
    if r.get("steps", "") != "":          steps.add(str(int(float(r["steps"]))))

summary = f"""# Milestone 2 — Summary (Template)

**Dataset**: Flickr30k (cleaned 5k pairs) → `data/processed/pairs.clean.csv`  
**Model**: {used_model}  
**Device / dtype**: {device} / {dtype}  
**Schedulers tested**: {", ".join(sorted(scheds)) if scheds else scheduler}  
**Guidance scales tested (CFG)**: {", ".join(sorted(cfgs)) if cfgs else "N/A"}  
**Steps tested**: {", ".join(sorted(steps)) if steps else "15"}  
**Early samples**: {len(selected)} images in `outputs/m2/submit/` (see `manifest.csv`)

## What we did
- Integrated CLIP text encoder with diffusion pipeline (`prompt_embeds` + `negative_prompt_embeds`).
- Baseline conditional generation (DDIM), CFG sweep, and scheduler comparison (DDIM / Euler-A / DPM-Solver).
- Logged runs to `metrics/m2_runs.csv`.

## Observations (fill in briefly)
- **CFG**: _low CFG → more diversity but weaker adherence; high CFG → better alignment but risk of artifacts/collapse._
- **Schedulers**: _Euler-A often sharper; DPM-Solver converges faster; DDIM stable but softer at same steps._
- **Failure modes**: _e.g., counting objects, text rendering, small details; high CFG may over-saturate._
- **Runtime (CPU)**: _kept steps small; fixed seed for fair comparison._

## Next (toward M3)
- Add quantitative metrics (FID/IS) on GPU; keep CLIPScore as a quick proxy.
- Parameter sensitivity grid (CFG × steps × scheduler) to select best trade-offs.
- Optional: lightweight LoRA finetune on a subset for better alignment.
"""
SUMMARY_MD.write_text(summary, encoding="utf-8")
print("Wrote 1-page summary template ->", SUMMARY_MD)
print("SUBMIT folder ready:", SUBMIT_DIR)

Selected 10 images (most recent) for submission.
Copied to: /scratch/qin.yife/Generative_Project/outputs/m2/submit
 - m2_01.png <- DPMSolver_CFG7.5_s15_seed20251114_02.png
 - m2_02.png <- DPMSolver_CFG7.5_s15_seed20251114_01.png
 - m2_03.png <- EulerA_CFG7.5_s15_seed20251114_02.png
 - m2_04.png <- EulerA_CFG7.5_s15_seed20251114_01.png
 - m2_05.png <- DDIM_CFG7.5_s15_seed20251114_02.png
 ...
Wrote manifest -> /scratch/qin.yife/Generative_Project/outputs/m2/submit/manifest.csv
Wrote 1-page summary template -> /scratch/qin.yife/Generative_Project/outputs/m2/M2_summary_template.md
SUBMIT folder ready: /scratch/qin.yife/Generative_Project/outputs/m2/submit


In [1]:
from pathlib import Path
import csv, re, shutil, math

# ---------- paths ----------
PROJECT     = Path("/scratch/qin.yife/Generative_Project")
M2_OUT_DIR  = PROJECT / "outputs" / "m2"
LOG_CSV     = PROJECT / "metrics" / "m2_runs.csv"
COMPARE_DIR = M2_OUT_DIR / "compare_by_prompt"
COMPARE_DIR.mkdir(parents=True, exist_ok=True)
GALLERY_MD  = COMPARE_DIR / "gallery.md"
PANEL_CSV   = COMPARE_DIR / "panel_manifest.csv"

# ---------- robust log reader (tolerate inconsistent columns) ----------
def load_log_rows(path: Path):
    rows = []
    if not path.exists():
        return rows
    with open(path, "r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f, restkey="__extra__", restval="")
        for r in reader:
            # normalize some fields
            r["file"] = (r.get("file") or "").strip()
            r["prompt"] = (r.get("prompt") or "").strip()
            r["scheduler"] = (r.get("scheduler") or "").strip()
            r["guidance_scale"] = (r.get("guidance_scale") or "").strip()
            r["steps"] = (r.get("steps") or "").strip()
            r["seed"] = (r.get("seed") or "").strip()
            r["via"]  = (r.get("via")  or "").strip()
            r["tag"]  = (r.get("tag")  or "").strip()
            rows.append(r)
    return rows

log_rows = load_log_rows(LOG_CSV)

# ---------- index by prompt ----------
# keep only rows that actually have an existing image file
valid = []
for r in log_rows:
    fp = Path(r["file"])
    if fp.suffix.lower() in {".png", ".jpg", ".jpeg"} and fp.exists():
        valid.append(r)

# group by prompt text
from collections import defaultdict
by_prompt = defaultdict(list)
for r in valid:
    if r["prompt"]:
        by_prompt[r["prompt"]].append(r)

# ---------- helpers ----------
def slugify(text: str, max_len: int = 64):
    # keep letters/numbers/space, collapse whitespace, hyphenate
    t = re.sub(r"[^A-Za-z0-9\s\-]+", "", text).strip()
    t = re.sub(r"\s+", " ", t)
    slug = t.lower().replace(" ", "-")
    if len(slug) > max_len:
        slug = slug[:max_len].rstrip("-")
    return slug if slug else "prompt"

def label_from_row(r: dict):
    # human-readable parameter label for filenames and captions
    sch = r["scheduler"] or "?"
    cfg = r["guidance_scale"] or "?"
    stp = r["steps"] or "?"
    via = r["via"] or ""
    tag = r["tag"] or ""
    extra = []
    if via: extra.append(via)
    if tag: extra.append(tag)
    extra_str = ("-" + "-".join(extra)) if extra else ""
    return f"{sch}_cfg{cfg}_s{stp}{extra_str}"

# ---------- build panels ----------
# limit per prompt to avoid huge folders (you can change this later)
MAX_PER_PROMPT = 18

panel_rows = []   # for global CSV
sections = []     # for gallery.md

prompt_ids = sorted(by_prompt.keys())
if not prompt_ids:
    raise RuntimeError("No valid (prompt, image) pairs found from the log. Make sure you ran M2 cells and logged runs.")

for idx, prompt in enumerate(prompt_ids, start=1):
    items = by_prompt[prompt]
    # sort by (scheduler, numeric cfg, steps, filename) for stable panels
    def to_float(x):
        try: return float(x)
        except: return math.inf
    items.sort(key=lambda r: (r["scheduler"], to_float(r["guidance_scale"]), to_float(r["steps"]), r["file"]))

    # cap to MAX_PER_PROMPT (keeps panels compact for the report)
    items = items[:MAX_PER_PROMPT]

    slug = f"p{idx:02d}_" + slugify(prompt, max_len=48)
    pdir = COMPARE_DIR / slug
    pdir.mkdir(parents=True, exist_ok=True)

    # copy images into the prompt folder with labeled names
    copied = []
    for j, r in enumerate(items, start=1):
        src = Path(r["file"])
        label = label_from_row(r)
        dst = pdir / f"{j:02d}_{label}{src.suffix.lower()}"
        shutil.copy2(src, dst)
        copied.append((src, dst, r))

        # collect for global CSV
        panel_rows.append({
            "prompt_id": f"{idx:02d}",
            "prompt": prompt,
            "prompt_slug": slug,
            "dst_name": dst.name,
            "src_path": str(src),
            "scheduler": r["scheduler"],
            "guidance_scale": r["guidance_scale"],
            "steps": r["steps"],
            "seed": r["seed"],
            "via": r["via"],
            "tag": r["tag"],
        })

    # build markdown section for this prompt
    sections.append(f"## {idx:02d}. {prompt}\n\n**Folder:** `{slug}`  \n")
    if copied:
        # make a small gallery table (3 per row)
        rows_md = []
        row = []
        for k, (_, dst, r) in enumerate(copied, start=1):
            cap = label_from_row(r)
            row.append(f'<div><img src="{slug}/{dst.name}" width="260"><br><sub>{cap}</sub></div>')
            if k % 3 == 0:
                rows_md.append("<p>" + " ".join(row) + "</p>")
                row = []
        if row:
            rows_md.append("<p>" + " ".join(row) + "</p>")
        sections.append("\n".join(rows_md))
    sections.append("\n---\n")

# write global panel manifest
with open(PANEL_CSV, "w", encoding="utf-8", newline="") as f:
    cols = ["prompt_id","prompt","prompt_slug","dst_name","src_path","scheduler","guidance_scale","steps","seed","via","tag"]
    w = csv.DictWriter(f, fieldnames=cols)
    w.writeheader()
    for r in panel_rows:
        w.writerow(r)

# write gallery markdown
header = (
    "# Milestone 2 — Prompt-wise Comparison Panels\n\n"
    f"- Root: `{COMPARE_DIR}`\n"
    f"- Global manifest: `{PANEL_CSV.name}`\n"
    "- Each section shows **the same prompt** with **different parameters** (scheduler/CFG/steps), filenames include settings.\n\n"
    "---\n"
)
GALLERY_MD.write_text(header + "\n".join(sections), encoding="utf-8")

print("Built comparison panels.")
print("Folders written under:", COMPARE_DIR)
print("Global panel manifest ->", PANEL_CSV)
print("Markdown gallery ->", GALLERY_MD)
print(f"Prompts grouped: {len(prompt_ids)} (capped {MAX_PER_PROMPT} images per prompt)")

Built comparison panels.
Folders written under: /scratch/qin.yife/Generative_Project/outputs/m2/compare_by_prompt
Global panel manifest -> /scratch/qin.yife/Generative_Project/outputs/m2/compare_by_prompt/panel_manifest.csv
Markdown gallery -> /scratch/qin.yife/Generative_Project/outputs/m2/compare_by_prompt/gallery.md
Prompts grouped: 4 (capped 18 images per prompt)
